# **Evaluacion de Data set**
Tecnicatura en inteligencia artificial

Universidad Nacional de Hurlingham

Docente: Andrea Rey

Alumno : Nicolas Seivane

Librerias que voy a utilizar


In [1]:
import pandas as pd
import sklearn as sk
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import StratifiedKFold

# .head()
# .info()
# .isnull().any()
# df.column.value_counts()
# .max()
# .min()
# .describe()

In [2]:
datos = pd.read_csv("C:\\Users\\User\\Documents\\Proyecto integrador\\cirrhosis\\cirrhosis.csv")
datos.drop_duplicates(inplace=True)
datos.dropna(inplace=True)
datos.drop(columns=["ID"], inplace=True)
#datos['NSP'] = datos['NSP'].replace({1: 'Normal', 2: 'Sospechoso', 3: 'Patologico'})
datos.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


In [3]:
datos.info()


<class 'pandas.core.frame.DataFrame'>
Index: 276 entries, 0 to 311
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   N_Days         276 non-null    int64  
 1   Status         276 non-null    object 
 2   Drug           276 non-null    object 
 3   Age            276 non-null    int64  
 4   Sex            276 non-null    object 
 5   Ascites        276 non-null    object 
 6   Hepatomegaly   276 non-null    object 
 7   Spiders        276 non-null    object 
 8   Edema          276 non-null    object 
 9   Bilirubin      276 non-null    float64
 10  Cholesterol    276 non-null    float64
 11  Albumin        276 non-null    float64
 12  Copper         276 non-null    float64
 13  Alk_Phos       276 non-null    float64
 14  SGOT           276 non-null    float64
 15  Tryglicerides  276 non-null    float64
 16  Platelets      276 non-null    float64
 17  Prothrombin    276 non-null    float64
 18  Stage          

In [4]:
datos.describe()

,N_Days,Age,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
count,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000
mean,1979.166667,18189.326087,3.333696,371.260870,3.516812,100.768116,1996.611594,124.119239,124.978261,261.771739,10.735507,3.039855
std,1112.380295,3843.555619,4.601074,234.788363,0.404789,88.268746,2115.477894,56.719952,65.280761,93.128857,1.008315,0.853999
min,41.000000,9598.000000,0.300000,120.000000,1.960000,4.000000,289.000000,28.380000,33.000000,62.000000,9.000000,1.000000
25%,1185.750000,15162.500000,0.800000,249.500000,3.310000,42.750000,922.500000,82.457500,85.000000,200.000000,10.000000,2.000000
50%,1788.000000,18156.500000,1.400000,310.000000,3.545000,74.000000,1277.500000,116.625000,108.000000,257.000000,10.600000,3.000000
75%,2689.750000,20667.500000,3.525000,401.000000,3.772500,129.250000,2068.250000,153.450000,151.250000,318.250000,11.200000,4.000000
max,4556.000000,28650.000000,28.000000,1775.000000,4.400000,588.000000,13862.400000,457.250000,598.000000,563.000000,17.100000,4.000000


In [5]:
n_filas = len(datos)

for column in datos.columns:
    unicos = datos[column].unique()
    n_unicos = len(unicos)
    
    # Si es categórica (tipo object/category) o es numérica con pocos únicos
    if (datos[column].dtype == 'object') or (len(unicos) <= 10):
        print(f"\nColumna '{column}' (candidata categórica): {n_unicos} valores únicos")
        
        
        for unico in unicos:
                count = len(datos[datos[column] == unico])
                normalizado = count / n_filas
                print(f"   Hay {normalizado:.2%} codificado en {unico}")
    else:
        print(f"\nColumna '{column}' tiene media: {datos[column].mean():.2f}, valor maximo: {datos[column].max():.2f} y valor minimo: {datos[column].min():.2f}")


Columna 'N_Days' tiene media: 1979.17, valor maximo: 4556.00 y valor minimo: 41.00

Columna 'Status' (candidata categórica): 3 valores únicos
   Hay 40.22% codificado en D
   Hay 53.26% codificado en C
   Hay 6.52% codificado en CL

Columna 'Drug' (candidata categórica): 2 valores únicos
   Hay 49.28% codificado en D-penicillamine
   Hay 50.72% codificado en Placebo

Columna 'Age' tiene media: 18189.33, valor maximo: 28650.00 y valor minimo: 9598.00

Columna 'Sex' (candidata categórica): 2 valores únicos
   Hay 87.68% codificado en F
   Hay 12.32% codificado en M

Columna 'Ascites' (candidata categórica): 2 valores únicos
   Hay 6.88% codificado en Y
   Hay 93.12% codificado en N

Columna 'Hepatomegaly' (candidata categórica): 2 valores únicos
   Hay 51.45% codificado en Y
   Hay 48.55% codificado en N

Columna 'Spiders' (candidata categórica): 2 valores únicos
   Hay 28.99% codificado en Y
   Hay 71.01% codificado en N

Columna 'Edema' (candidata categórica): 3 valores únicos
   Hay 

In [6]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        print(column, datos[column].unique())



Status ['D' 'C' 'CL']
Drug ['D-penicillamine' 'Placebo']
Sex ['F' 'M']
Ascites ['Y' 'N']
Hepatomegaly ['Y' 'N']
Spiders ['Y' 'N']
Edema ['Y' 'N' 'S']


In [7]:
for column in datos.columns:
    if len(datos[column].unique()) <= 10 and datos[column].dtype == 'object':
        unicos = datos[column].unique()
        for i in range(len(unicos)):
            datos[column] = datos[column].replace({unicos[i]: i})
        print(column, datos[column].unique())

Status [0 1 2]
Drug [0 1]
Sex [0 1]
Ascites [0 1]
Hepatomegaly [0 1]
Spiders [0 1]
Edema [0 1 2]


C:\Users\User\AppData\Local\Temp\ipykernel_16480\495973239.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos[column] = datos[column].replace({unicos[i]: i})


In [8]:
for column in datos.columns:
    if len(datos[column].unique()) > 4:
        datos[column] = (datos[column] - datos[column].mean()) / datos[column].std()

In [9]:
atributos = datos.columns[:-1]
concepto = datos.columns[-1]

datos.head()

,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,-1.419628,0,0,0.851991,0,0,0,0,0,2.426891,-0.469618,-2.264915,0.625724,-0.131701,0.243843,0.720300,-0.770671,1.452417,4.0
1,2.266161,1,0,0.631622,0,1,0,0,1,-0.485473,-0.294993,1.539540,-0.529838,2.551758,-0.186870,-0.566450,-0.437799,-0.134390,3.0
2,-0.869457,0,0,1.926517,1,1,1,1,2,-0.420271,-0.831646,-0.090940,1.237492,-0.699895,-0.493993,-1.071958,-1.189446,1.254066,4.0
3,-0.048694,0,0,0.469532,0,1,0,0,2,-0.333334,-0.542024,-2.413140,-0.416547,1.950003,-1.119346,-0.505176,-0.845836,-0.431916,4.0
4,-0.427162,2,1,-1.111296,0,1,0,0,1,0.014411,-0.392953,0.032581,0.478447,-0.626625,-0.193393,-0.811545,-1.350513,0.163136,3.0


In [10]:
def validacion_cruzada(conjunto, atributos, concepto, k=5, random=False, agregar_unos=False, undersample=False, oversample=False):
  
  if undersample:
    cantidad_menor_concepto = conjunto[concepto].value_counts().min()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_menor_concepto, random_state=42) for valor in conjunto[concepto].unique()])

  if oversample:
    cantidad_mayor_concepto = conjunto[concepto].value_counts().max()
    conjunto = pd.concat([conjunto[conjunto[concepto] == valor].sample(n=cantidad_mayor_concepto, replace=True, random_state=42) for valor in conjunto[concepto].unique()])

  if random: conjunto = conjunto.sample(frac=1, random_state=11).reset_index(drop=True)

  x = conjunto[atributos]
  if agregar_unos: x.insert(0, "Unos", 1)

  y = conjunto[concepto]

  resultados = []
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

  for train_index, test_index in skf.split(x, y):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        resultados.append((x_train, y_train, x_test, y_test))

  return resultados

from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)

from sklearn.naive_bayes import GaussianNB

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


from sklearn.svm import SVC

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)

from sklearn.linear_model import LogisticRegression

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    matthews_corrcoef,
    balanced_accuracy_score,
    log_loss,
    cohen_kappa_score
)

    y_true_total.extend(y_test)
    y_pred_total.extend(y_pred)

print("Matriz de confusión total:\n", confusion_matrix(y_true_total, y_pred_total))

LogisticRegression(
    penalty="l2",          # regularización: "l1", "l2", "elasticnet", None
    C=1.0,                 # inverso de la fuerza de regularización (menor C = más regularización)
    solver="lbfgs",        # optimizador ("lbfgs", "saga", "liblinear", "newton-cg")
    max_iter=1000,         # iteraciones máximas (importante en datasets grandes)
    multi_class="multinomial",    # "ovr" (uno-vs-rest) o "multinomial"
    random_state=42
)

# *Regresion Logistica*

In [11]:
grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l1","l2","elasticnet", None],  # 'l1' si usas solver='liblinear' o 'saga'
    "solver": ["lbfgs","saga", "newton-cg"],# según el penalty que uses, liblinear no soporta None, newton-cg no soporta l1 ni elasticnet
    "multi_class": ["ovr", "multinomial"] # si es multiclass
}
 
## cohen_kappa_score,     accuracy_score,    precision_score,     recall_score,    f1_score,    roc_auc_score,


#with open("regresion_logistica_resultados.txt", "w") as f:
with open("regresion_logistica_resultados_cirrosis.txt", "w") as f:
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds):\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for multi_class in grid["multi_class"]:
                for solver in grid["solver"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                #"roc_auc": 0
                            }
                    try:
                        inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)
                        
                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                #probs = modelo.predict_proba(x_test)
                                #scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                
                        fin = time.time()
                        tiempo_total = fin - inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds) 
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Regresion Logistica con validacion cruzada (5 folds) undersample:\n")
    
    for C in grid["C"]:

        for penalty in grid["penalty"]:
            for solver in grid["solver"]:
                for multi_class in grid["multi_class"]:
                    # chequeo de compatibilidad
                    if penalty == "l1" and solver not in ["liblinear", "saga"]:
                        continue
                    if penalty == "elasticnet" and solver != "saga":
                        continue
                    if penalty is None and solver not in ["lbfgs","newton-cg","sag","saga"]:
                        continue
                    scoring_funcs = {
                                "cohen_kappa": 0,
                                "accuracy": 0,
                                "precision": 0,
                                "recall": 0,
                                "f1": 0,
                                #"roc_auc": 0
                            }
                    try:
                        tiempo_inicio = time.time()
                        modelo = LogisticRegression(
                            C=C,
                            penalty=penalty,
                            solver=solver,
                            l1_ratio=0.5 if penalty=="elasticnet" else None,
                            multi_class=multi_class,
                            max_iter=1000,
                            random_state=42
                        )

                        folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                        for x_train, y_train, x_test, y_test in folds:
                                modelo.fit(x_train, y_train)
                                pred = modelo.predict(x_test)
                                scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                #probs = modelo.predict_proba(x_test)
                                #scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")


                        fin = time.time()
                        tiempo_total = fin - tiempo_inicio
                        for key in scoring_funcs:
                            scoring_funcs[key] /= len(folds)  # Promediar las métricas

                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                    except Exception as e:
                        f.write(f"\nC={C}, penalty={penalty}, solver={solver}, multi_class={multi_class} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.w

RandomForestClassifier(
    n_estimators=100,      # número de árboles en el bosque
    criterion="gini",      # función de calidad de la división ("gini" o "entropy")
    max_depth=None,        # profundidad máxima de cada árbol
    min_samples_split=2,   # min muestras para dividir un nodo
    min_samples_leaf=1,    # min muestras en una hoja
    max_features="sqrt",   # nº de features a considerar en cada split ("sqrt", "log2", None)
    bootstrap=True,        # si usar bootstrap samples para entrenar cada árbol
    random_state=42,       # semilla
    n_jobs=-1              # usa todos los cores
)


# *Random Forest*

In [12]:
grid = {
    "n_estimators": [50, 100, 200, 500, 1000],
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 3, 5, 7, 9],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": [None, "sqrt", "log2"]
}



#with open("Random_forest.txt", "w") as f:
with open("Random_forest__cirrosis.txt", "w") as f:
    f.write("Resultados de Random Forest con validacion cruzada (5 folds):\n")

    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    #"roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    #probs = modelo.predict_proba(x_test)
                                    #if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        #scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    #else:
                                        # multiclase → usar esquema OVR
                                        #scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Random Forest con validacion cruzada (5 folds) undersample:\n")
    
    for criterion in grid["criterion"]:
        for max_depth in grid["max_depth"]:
            for min_samples_split in grid["min_samples_split"]:
                for min_samples_leaf in grid["min_samples_leaf"]:
                    for max_features in grid["max_features"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    #"roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = RandomForestClassifier(
                                criterion=criterion,
                                max_depth=max_depth,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                max_features=max_features,
                                random_state=42
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    #probs = modelo.predict_proba(x_test)
                                    #if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                        #scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    #else:
                                        # multiclase → usar esquema OVR
                                        #scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} "
                                    f"--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")
                        except Exception as e:
                            f.write(f"\ncriterion={criterion}, max_depth={max_depth}, min_samples_split={min_samples_split}, "
                                    f"min_samples_leaf={min_samples_leaf}, max_features={max_features} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

GaussianNB(
    var_smoothing=1e-9     # suavizado para evitar divisiones por cero
)


# *Naive Bayes*

In [13]:
grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]  
}

#with open("Naiva_Bayes.txt", "w") as f:
with open("Naiva_Bayes_cirrosis.txt", "w") as f:
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds):\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    #"roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    #probs = modelo.predict_proba(x_test)
                                    #if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    #else:
                                        # multiclase
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de Naive Bayes con validacion cruzada (5 folds) undersample:\n")

    for var_smoothing in grid["var_smoothing"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    #"roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = GaussianNB(var_smoothing=var_smoothing)
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    #probs = modelo.predict_proba(x_test)
                                    #if len(set(y_test)) == 2:
                                        # binaria → prob de la clase positiva
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    #else:
                                        # multiclase
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\n var_smoothing={var_smoothing}--> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\n var_smoothing={var_smoothing}--> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag

SVC(
    C=1.0,                 # penalización del error (mayor C = menos margen, más sobreajuste)
    kernel="rbf",          # kernel: "linear", "poly", "rbf", "sigmoid"
    degree=3,              # grado del polinomio (si kernel="poly")
    gamma="scale",         # controla la influencia de un solo ejemplo ("scale", "auto" o un valor numérico)
    probability=True,     # si calcular probabilidades (más lento)
    random_state=42
)


# *SVM*

In [14]:
grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 15, 20, 25],
    "kernel": ["linear", "poly", "rbf", "sigmoid"],
    "gamma": ["scale", "auto"],
    "degree": [2,3,4,5,6,7,8,9,10]
}

#with open("SVM.txt", "w") as f:
with open("SVM__cirrosis.txt", "w") as f:
    f.write("Resultados de SVM con validacion cruzada (5 folds):\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                degrees_to_use = grid["degree"] if kernel == "poly" else [3]
                for degree in degrees_to_use:
                        scoring_funcs = {
                                "cohen_kappa": 0,
                                 "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    #"roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )

                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    #probs = modelo.predict_proba(x_test)

                                    #if len(set(y_test)) == 2:
                                        # binaria
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    #else:
                                        # multiclase
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, kernel={kernel}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")
    f.write("\n\n")
    f.write("------------------------------------------------------\n\n")
    f.write("Resultados de SVM con validacion cruzada (5 folds) undersample:\n")

    for C in grid["C"]:
        for kernel in grid["kernel"]:
            for gamma in grid["gamma"]:
                if kernel != "poly" and "degree" in grid:
                    continue
                for degree in grid["degree"]:
                        scoring_funcs = {
                                    "cohen_kappa": 0,
                                    "accuracy": 0,
                                    "precision": 0,
                                    "recall": 0,
                                    "f1": 0,
                                    #"roc_auc": 0
                                }
                        try:
                            inicio = time.time()
                            modelo = SVC(
                                  C=C,
                                    kernel=kernel,
                                    gamma=gamma,
                                    degree=degree if kernel=="poly" else None,
                                    max_iter=1000,
                                    random_state=42,
                                    probability=True
                            )
                            folds = validacion_cruzada(datos, atributos, concepto, k=5, random=True, undersample=True)

                            for x_train, y_train, x_test, y_test in folds:
                                    modelo.fit(x_train, y_train)
                                    pred = modelo.predict(x_test)
                                    scoring_funcs["cohen_kappa"] += cohen_kappa_score(y_test, pred)
                                    scoring_funcs["accuracy"] += accuracy_score(y_test, pred)
                                    scoring_funcs["precision"] += precision_score(y_test, pred, average="weighted")
                                    scoring_funcs["recall"] += recall_score(y_test, pred, average="weighted")
                                    scoring_funcs["f1"] += f1_score(y_test, pred, average="weighted")
                                    #probs = modelo.predict_proba(x_test)

                                    #if len(set(y_test)) == 2:
                                        # binaria
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs[:, 1])
                                    #else:
                                        # multiclase
                                     #   scoring_funcs["roc_auc"] += roc_auc_score(y_test, probs, multi_class="ovr")

                                    
                            fin = time.time()
                            tiempo_total = fin - inicio
                            for key in scoring_funcs:
                                scoring_funcs[key] /= len(folds) 
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> {scoring_funcs}, tiempo_total={tiempo_total:.2f} segundos\n")

                        except Exception as e:
                            f.write(f"\nC={C}, C={C}, gamma={gamma}, degree={degree} --> ERROR: {str(e)}\n")

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(averag